In [1]:
import os
import ast
import sys
import json
import torch
import pickle
import subprocess

sys.path.append('../')

import pandas as pd

from pathlib import Path
from tqdm import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report
#from utils.post_processing import post_process


In [2]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

In [3]:
CURRENT_DIR = Path.cwd()
#FT_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning"
FT_DIR = CURRENT_DIR / "emotion_analysis_comics" / "comics_FT"
#DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning" / "datasets"
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "comics_FT" / "datasets"

ERC_DIR = FT_DIR.parent
LLAMA_FACTORY_DIR = ERC_DIR / "LLaMA-Factory"

BASE_MODEL = "unsloth/llama-3-8b-Instruct-bnb-4bit"
LOGGING_DIR = FT_DIR / "training_logs"
OUTPUT_DIR = FT_DIR / "saved_models" / f"""comics35_pg_noscene_{BASE_MODEL.split("/")[1]}"""

In [4]:
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "comics_FT" / "datasets"

In [5]:
train_dataset_name = f"""comics35_utterance_pg_random_train.json"""
test_dataset_name = f"""comics35_utterance_pg_random_test.json"""

train_dataset_file = DATASET_DIR / train_dataset_name
test_dataset_file = DATASET_DIR / test_dataset_name

In [6]:
train_dataset_file

PosixPath('/Utilisateurs/umushtaq/emotion_analysis_comics/comics_FT/datasets/comics35_utterance_pg_random_train.json')

In [7]:
if not os.path.exists(os.path.join(FT_DIR, "model_args")):
    os.mkdir(os.path.join(FT_DIR, "model_args"))

train_file = FT_DIR / "model_args" / f"""{train_dataset_name.split(".")[0].split("train")[0]}{BASE_MODEL.split("/")[1]}.json"""

In [8]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}

In [9]:
with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "r") as jsonFile:
    data = json.load(jsonFile)

data["comics"] = dataset_info_line

with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "w") as jsonFile:
    json.dump(data, jsonFile)

In [10]:
NB_EPOCHS = 20

In [11]:
args = dict(
    
  stage="sft",                           # do supervised fine-tuning
  do_train=True,

  model_name_or_path=BASE_MODEL,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  num_train_epochs=NB_EPOCHS,            # the epochs of training
  output_dir=str(OUTPUT_DIR),                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents

  dataset="comics",                      # dataset name
  template="llama3",                     # use llama3 prompt template
  train_on_prompt=True,
  val_size=0.1,
  max_samples=10000,                       # use 500 examples in each dataset

  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  per_device_train_batch_size=8,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="linear",            # use cosine learning rate scheduler
  loraplus_lr_ratio=32.0,                # use LoRA+ algorithm with lambda=16.0
  #temperature=0.5,
  
  warmup_ratio=0.1,                      # use warmup scheduler    
  learning_rate=5e-5,                    # the learning rate
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  
  fp16=True,                             # use float16 mixed precision training
  quantization_bit=4,                    # use 4-bit QLoRA  
  #use_liger_kernel=True,
  #quantization_device_map="auto",
  
  #load_best_model_at_end=True,
  #metric_for_best_model="eval_loss",
  #save_strategy="epoch",
  #eval_strategy="epoch",
  logging_steps=10,                      # log every 10 steps
  save_steps=5000,                       # save checkpoint every 1000 steps    
  logging_dir=str(LOGGING_DIR),
  
  #use_unsloth=True,
  report_to="tensorboard"                       # discards wandb

)

In [12]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [13]:
p = subprocess.Popen(["llamafactory-cli", "train", train_file], cwd=LLAMA_FACTORY_DIR)

In [14]:
p.wait()

02/24/2025 15:09:02 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
02/24/2025 15:09:02 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16


[INFO|configuration_utils.py:672] 2025-02-24 15:09:03,018 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/fd5a4dc328319c1cfe9489eccfb9c6406bdfd469/config.json
[INFO|configuration_utils.py:739] 2025-02-24 15:09:03,019 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

02/24/2025 15:09:04 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>
02/24/2025 15:09:04 - INFO - llamafactory.data.loader - Loading dataset /Utilisateurs/umushtaq/emotion_analysis_comics/comics_FT/datasets/comics35_utterance_pg_random_train.json...


Generating train split: 691 examples [00:00, 5563.02 examples/s]
Running tokenizer on dataset: 100%|██████████| 691/691 [00:01<00:00, 404.50 examples/s]


training example:
input_ids:
[128000, 128006, 882, 128007, 271, 14711, 5867, 6082, 18825, 33257, 15766, 271, 2675, 527, 459, 11084, 20356, 6492, 6335, 58394, 304, 20303, 2363, 21976, 23692, 13, 4718, 3465, 374, 311, 24564, 22256, 3095, 323, 10765, 872, 14604, 2262, 382, 30521, 512, 12, 1472, 690, 5371, 264, 1160, 315, 22256, 3095, 505, 264, 2199, 304, 264, 20303, 2363, 198, 12, 578, 22256, 685, 1253, 3237, 832, 477, 5361, 21958, 271, 66913, 512, 16, 13, 10852, 3725, 24564, 279, 14604, 2317, 323, 16630, 315, 1855, 22256, 685, 304, 279, 2199, 198, 17, 13, 65647, 8581, 21958, 505, 279, 2768, 6989, 512, 256, 330, 4091, 498, 330, 4338, 70, 592, 498, 330, 69, 686, 498, 330, 83214, 2136, 498, 330, 20370, 9868, 498, 330, 4215, 498, 330, 60668, 702, 18, 13, 1789, 1855, 22256, 685, 304, 264, 20303, 2199, 11, 10765, 682, 21958, 3118, 323, 471, 459, 1358, 315, 20356, 18893, 304, 2015, 382, 93016, 50, 512, 16, 13, 5560, 27785, 279, 9382, 10212, 3485, 198, 17, 13, 9442, 28832, 7354, 264, 832, 8614, 

...

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
[INFO|trainer.py:667] 2025-02-24 15:09:11,912 >> Using auto half precision backend


02/24/2025 15:09:12 - INFO - llamafactory.train.trainer_utils - Using LoRA+ optimizer with loraplus lr ratio 32.00.


[INFO|trainer.py:2243] 2025-02-24 15:09:12,826 >> ***** Running training *****
[INFO|trainer.py:2244] 2025-02-24 15:09:12,826 >>   Num examples = 621
[INFO|trainer.py:2245] 2025-02-24 15:09:12,826 >>   Num Epochs = 20
[INFO|trainer.py:2246] 2025-02-24 15:09:12,826 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:2249] 2025-02-24 15:09:12,826 >>   Total train batch size (w. parallel, distributed & accumulation) = 32
[INFO|trainer.py:2250] 2025-02-24 15:09:12,826 >>   Gradient Accumulation steps = 4
[INFO|trainer.py:2251] 2025-02-24 15:09:12,826 >>   Total optimization steps = 380
[INFO|trainer.py:2252] 2025-02-24 15:09:12,832 >>   Number of trainable parameters = 20,971,520
  0%|          | 0/380 [00:00<?, ?it/s]/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autoc

{'loss': 1.7049, 'grad_norm': 0.6366852521896362, 'learning_rate': 1.3157894736842106e-05, 'epoch': 0.51}
{'loss': 0.8897, 'grad_norm': 0.4830089211463928, 'learning_rate': 2.6315789473684212e-05, 'epoch': 1.03}
{'loss': 0.6819, 'grad_norm': 0.26245155930519104, 'learning_rate': 3.9473684210526316e-05, 'epoch': 1.54}
{'loss': 0.6042, 'grad_norm': 0.1603444665670395, 'learning_rate': 4.970760233918128e-05, 'epoch': 2.05}
{'loss': 0.5737, 'grad_norm': 0.3750247657299042, 'learning_rate': 4.824561403508772e-05, 'epoch': 2.56}
{'loss': 0.5282, 'grad_norm': 0.26157352328300476, 'learning_rate': 4.678362573099415e-05, 'epoch': 3.08}
{'loss': 0.4915, 'grad_norm': 0.1739841252565384, 'learning_rate': 4.5321637426900585e-05, 'epoch': 3.59}
{'loss': 0.4728, 'grad_norm': 0.1984020620584488, 'learning_rate': 4.3859649122807014e-05, 'epoch': 4.1}
{'loss': 0.4266, 'grad_norm': 0.21445780992507935, 'learning_rate': 4.239766081871345e-05, 'epoch': 4.62}
{'loss': 0.3938, 'grad_norm': 0.2217944562435150

0

In [15]:
args = dict(
  model_name_or_path=BASE_MODEL, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=str(OUTPUT_DIR),            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)

In [16]:
model = ChatModel(args)

[INFO|configuration_utils.py:672] 2025-02-24 15:59:08,892 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/fd5a4dc328319c1cfe9489eccfb9c6406bdfd469/config.json
[INFO|configuration_utils.py:739] 2025-02-24 15:59:08,893 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

02/24/2025 15:59:10 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


[INFO|configuration_utils.py:672] 2025-02-24 15:59:10,728 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/fd5a4dc328319c1cfe9489eccfb9c6406bdfd469/config.json
[INFO|configuration_utils.py:739] 2025-02-24 15:59:10,729 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

02/24/2025 15:59:10 - WARNING - llamafactory.model.model_utils.quantization - `quantization_bit` will not affect on the PTQ-quantized models.
02/24/2025 15:59:10 - INFO - llamafactory.model.model_utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.
02/24/2025 15:59:10 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


[WARNING|quantization_config.py:400] 2025-02-24 15:59:10,917 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
[INFO|modeling_utils.py:3726] 2025-02-24 15:59:11,268 >> loading weights file model.safetensors from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/fd5a4dc328319c1cfe9489eccfb9c6406bdfd469/model.safetensors
[INFO|modeling_utils.py:1622] 2025-02-24 15:59:11,313 >> Instantiating LlamaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1099] 2025-02-24 15:59:11,318 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "pad_token_id": 128255
}

[INFO|quantizer_bnb_4bit.py:122] 2025-02-24 15:59:11,472 >> target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization
[INFO|modeling_utils.py:4568] 2025-02-24 15:

02/24/2025 15:59:13 - INFO - llamafactory.model.model_utils.attention - Using torch SDPA for faster training and inference.
02/24/2025 15:59:14 - INFO - llamafactory.model.adapter - Loaded adapter(s): /Utilisateurs/umushtaq/emotion_analysis_comics/comics_FT/saved_models/comics35_pg_noscene_llama-3-8b-Instruct-bnb-4bit
02/24/2025 15:59:14 - INFO - llamafactory.model.loader - all params: 8,051,232,768


[WARNING|logging.py:328] 2025-02-24 15:59:26,183 >> Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [17]:
model.engine

In [18]:
model.engine_type

'huggingface'

In [19]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append(sample["instruction"] + sample["input"])
    #test_prompts.append(sample["input"])
    test_grounds.append(sample["output"])

In [20]:
test_predictions = []

In [21]:
messages = []

for prompt in tqdm(test_prompts, desc="Running inferences ..."):
    #print(type(prompt))
    #messages.append({"role": "user", "content": prompt})
    message = [{"role": "user", "content": prompt}]
    test_predictions.append(model.chat(message))
    #test_predictions.append(model.chat(prompt))

Running inferences ...: 100%|██████████| 183/183 [10:09<00:00,  3.33s/it]


In [22]:
# batch_size = 1  # Adjust batch size according to memory and model limitations

# # Initialize list to hold predictions
# test_predictions = []

# # Iterate over test_prompts in batches
# for i in tqdm(range(0, len(test_prompts), batch_size), desc="Running inferences ..."):
#     # Create a batch of messages
#     batch_prompts = test_prompts[i:i + batch_size]
    
#     # Prepare the messages for the batch
#     batch_messages = [[{"role": "user", "content": prompt}] for prompt in batch_prompts]
    
#     # Perform inference on the batch and store predictions
#     for message in batch_messages:
#         prediction = model.chat(message)
#         test_predictions.append(prediction)  # Collect batch predictions

In [23]:
len(test_predictions)

183

In [24]:
test_predictions

[[Response(response_text='{"emotions": [["anger"], ["anger"], ["fear"], ["fear"], ["anger"], ["fear"], ["anger"], ["fear", "surprise"], ["anger"], ["anger"]]}', response_length=44, prompt_length=706, finish_reason='stop')],
 [Response(response_text='{"emotions": [["neutral"], ["neutral"], ["anger", "surprise"], ["anger", "sadness"], ["sadness"], ["sadness"], ["anger"], ["anger"], ["surprise"], ["anger", "sadness"], ["sadness", "surprise"], ["sadness", "surprise"], ["anger", "sadness"], ["anger", "sadness"]]}', response_length=81, prompt_length=906, finish_reason='stop')],
 [Response(response_text='{"emotions": [["joy"], ["joy"], ["anger"], ["sadness"], ["fear"], ["anger"], ["anger"], ["sadness"], ["anger"], ["anger"]]}', response_length=39, prompt_length=978, finish_reason='stop')],
 [Response(response_text='{"emotions": [["fear"], ["fear"], ["sadness"], ["sadness"], ["fear"], ["fear"], ["fear"], ["fear"]]}', response_length=38, prompt_length=938, finish_reason='stop')],
 [Response(res

In [25]:
processed_preds = []
bad_idx = []

for i, raw_pred in enumerate(test_predictions):
    try:
        x = json.loads(raw_pred[0].response_text)["emotions"]
        processed_preds.append(x)
    except:
        print(i)
        bad_idx.append(i)

123


In [26]:
# bad_idx.sort(reverse=True)

# # Remove elements from 'grounds' at the specified indices
# for idx in bad_idx:
    
#     #del processed_grounds[idx]
#     del processed_preds[idx]

In [27]:
len(processed_preds)

182

In [28]:
processed_grounds = []

for ground in test_grounds:
    x = json.loads(ground)["emotions"]
    processed_grounds.append(x)

In [29]:
len(processed_grounds)

183

In [30]:
bad_idx

[123]

In [31]:
processed_grounds = [item for i, item in enumerate(processed_grounds) if i not in bad_idx]

In [34]:
processed_grounds

[[['anger'],
  ['anger'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['sadness'],
  ['anger'],
  ['surprise'],
  ['surprise'],
  ['fear', 'surprise']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['anger', 'fear', 'surprise'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['anger'],
  ['anger'],
  ['surprise'],
  ['fear'],
  ['anger', 'surprise'],
  ['anger'],
  ['anger', 'sadness'],
  ['anger']],
 [['joy'],
  ['joy'],
  ['fear'],
  ['joy'],
  ['fear', 'sadness'],
  ['anger', 'disgust', 'surprise'],
  ['anger', 'disgust'],
  ['fear'],
  ['anger', 'disgust'],
  ['fear', 'surprise']],
 [['fear'],
  ['sadness'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['fear'],
  ['fear'],
  ['sadness']],
 [['joy'],
  ['joy'],
  ['sadness'],
  ['surprise'],
  ['sadness'],
  ['surprise'],
  ['fear', 'surprise'],
  ['joy'],
  ['joy'],
  ['sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'joy'],
  ['sadness', 'joy']],
 [['fear'],
  ['surprise', 'joy'],
  ['fear'],
  ['surprise'],
  ['fea

In [32]:
processed_preds

[[['anger'],
  ['anger'],
  ['fear'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['anger'],
  ['fear', 'surprise'],
  ['anger'],
  ['anger']],
 [['neutral'],
  ['neutral'],
  ['anger', 'surprise'],
  ['anger', 'sadness'],
  ['sadness'],
  ['sadness'],
  ['anger'],
  ['anger'],
  ['surprise'],
  ['anger', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['anger', 'sadness'],
  ['anger', 'sadness']],
 [['joy'],
  ['joy'],
  ['anger'],
  ['sadness'],
  ['fear'],
  ['anger'],
  ['anger'],
  ['sadness'],
  ['anger'],
  ['anger']],
 [['fear'],
  ['fear'],
  ['sadness'],
  ['sadness'],
  ['fear'],
  ['fear'],
  ['fear'],
  ['fear']],
 [['sadness'],
  ['joy'],
  ['sadness'],
  ['surprise'],
  ['surprise'],
  ['sadness'],
  ['sadness'],
  ['sadness'],
  ['joy'],
  ['joy'],
  ['sadness'],
  ['sadness'],
  ['joy']],
 [['fear', 'surprise'],
  ['surprise'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear', 'surprise'],
  ['fear', 'surprise'],
  ['fear', 'surprise'],
  ['anger'],
  ['sur

In [33]:
import ast

In [35]:
#processed_grounds = [ast.literal_eval(elem) for elem in processed_grounds]

In [36]:
#processed_preds = [ast.literal_eval(elem) for elem in processed_preds]

In [37]:
bad_idx = []

for idx, (i,j) in enumerate(zip(processed_grounds, processed_preds)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

5 11 10
16 1 8
28 14 13
62 19 18
76 2 3
121 7 8


In [38]:
processed_grounds = [item for i, item in enumerate(processed_grounds) if i not in bad_idx]
processed_preds = [item for i, item in enumerate(processed_preds) if i not in bad_idx]

In [40]:
len(processed_grounds), len(processed_preds)

(176, 176)

In [41]:
grounds = [item for sublist in processed_grounds for item in sublist]
predictions = [item for sublist in processed_preds for item in sublist]

In [42]:
len(grounds), len(predictions)

(1387, 1387)

In [43]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [44]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['j', 'o', 'protest', 'y'] will be ignored
  warnings.warn(


In [45]:
y_true_mhot.shape, y_pred_mhot.shape

((1387, 7), (1387, 7))

In [46]:
class_labels = mlb.classes_

In [47]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [48]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.528     0.586     0.555       485
     disgust      0.265     0.158     0.198        57
        fear      0.520     0.437     0.475       350
         joy      0.552     0.395     0.461       334
     neutral      0.132     0.093     0.109        75
     sadness      0.467     0.528     0.495       356
    surprise      0.526     0.503     0.515       316

   micro avg      0.500     0.472     0.486      1973
   macro avg      0.427     0.386     0.401      1973
weighted avg      0.497     0.472     0.480      1973
 samples avg      0.515     0.487     0.477      1973



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# with open({Path(FT_DIR)} / "classification_report.pickle", 'wb') as fh:
    
#      pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)